# 하나의 사업자 등록번호로 7개의 카드사의 가맹점 번호를 캡쳐하는 코드
# GUI로 작성 되었음.

# 알고리즘
### 1. 캡쳐를 저장할 '캡쳐'라는 폴더를 생성 
### 2. 일자별 정리를 위해 당일 날짜의 폴더 생성
### 3. gui폼 과 크롬창이 열립니다.
### 4. text칸에 사업자 등록번호를 작성후 확인 버튼을 누릅니다.
### 5. 크롬으로 bc카드 가맹점 사이트에 접속하게 한뒤 가맹점명 정보를 크롤링 한뒤 가맹점명 폴더를 생성합니다.
### 7. 가맹점번호를 캡쳐를 한뒤 생성해두었던 폴더에 저장합니다,
### 8. 삼성,신한,하나,롯데,신한,국민,현대,농협 가맹점 사이트순으로 캡쳐를 진행합니다.

### ※한번에 여러개의 사업자 등록번호를 캡쳐할수있습니다 text창에 '.'으로 사업자등록번호를 분리시키면 됩니다.
### ※gui폼으로 만들어서 pyinstaller를 이용하여 exe파일로 소장하여도 됩니다.
### ※C드라이브에 크롬웹드라이버를 넣어주셔야합니다.

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import os  
import datetime
import sys
from PyQt5.QtWidgets import *
from PyQt5.QtCore import *

#현재 날짜를 받아오는 변수생성
now = datetime.datetime.now()
nowdate = now.strftime('%Y.%m.%d')

#폴더명을 캡쳐로 할것입니다.
dir_path = '캡쳐'
dir_name = nowdate

# 캡쳐라는 폴더가 존재하는지 여부 판단
#있으면 None
#없으면 폴더생성
if(os.path.isdir(os.getcwd()+'/캡쳐')):
    None
else:
    os.mkdir(os.getcwd()+'/캡쳐')

#현재날짜의 폴더가 존재하는지 여부 판단
#있으면 None
#없으면 폴더생성
if(os.path.isdir(dir_path+'/'+dir_name)):
    None
else:
    os.mkdir(dir_path+'/'+dir_name+'/')


#크롬창을 킨다.
options = Options()
path = 'C:\chromedriver.exe'
driver = webdriver.Chrome(path,chrome_options=options)
driver.set_window_size(1800,900)



#클래스 생성
class MyWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setupUI()
        
        

    def setupUI(self):
        
        #gui폼을 위한 코드 입니다.
        self.setGeometry(600, 400, 200, 100)
        self.setWindowTitle("가맹점 번호 조회")

        # Label
        label = QLabel("번호", self)
        label.move(20, 20)

        # 사업자 등록번호를 넣을 칸입니다.
        self.lineEdit = QLineEdit("", self)
        self.lineEdit.move(80, 20)

        # 확인버튼 생성
        btn2 = QPushButton("확인", self)
        btn2.move(55, 60)
        btn2.resize(50,30)
        
        #엔터키를 누르거나 확인 버튼을 클릭하면 num_search라는 함수를 실행합니다
        
        self.lineEdit.returnPressed.connect(self.num_search)
        btn2.clicked.connect(self.num_search)
        
        #취소버튼 생성
        btn1 = QPushButton("취소", self)
        btn1.move(105, 60)
        btn1.resize(50,30)
        btn1.clicked.connect(self.close)
        
        # 콤보 박스를 생성합니다
        # BC = bc카드만 접속해서 캡쳐합니다
        #우편 = bc,삼성,신한,하나,롯데 카드만 접속해서 캡쳐합니다
        # ALL = bc,삼성,신한,하나,롯데,국민,현대 카드를 접속해서 캡쳐합니다
        self.statusBar = QStatusBar(self)
        self.setStatusBar(self.statusBar)
        self.cb=QComboBox(self)
        self.cb.addItem('All')
        self.cb.addItem('BC')
        self.cb.addItem('우편')
        self.cb.resize(50,20)   

        #카드사에 접속하여서 사업자 등록번호를 입력시키고 캡쳐하는 함수입니다.
    def num_search(self):
        #text창에 적었던 사업자 등록번호를 num에 받아 옵니다.
        num=self.lineEdit.text()
        
        #콤보박스가 BC일때 작동
        if(self.cb.currentText()=='BC'):
            #모든 사업자 등록 번호는 10자리입니다. 10자리 미만이면 작동 하지 않습니다.
            if(len(num)<10):
                QMessageBox.about(self,"","사업자 번호가 10자리 미만입니다.")
            else:   
                #작성한 사업자번호가 다를수 있기 때문에 try except문을 이용해서 예외처리를 했습니다 . 
                try:
                    #ok버튼을 누르면 캡쳐한다는 공고를 줍니다.
                    QMessageBox.about(self,"","OK 버튼을 누르면 캡쳐를 시작합니다.")
                    # 사업자 등록번호를 . 으로 분리 시켜놓으면 한번에 많은 가맹점번호를 캡쳐 할수있습니다.
                    b_num=num.split('.')
                    b_num
                    c=[]
                    
                    #bc카드 접속
                    driver.get('https://www.bccard.com/app/merchant/StoreNoInqActn.do')
                    # 가맹점번호가 보이게하기위해 스크롤을 내립니다.
                    driver.execute_script("window.scrollTo(0,140);")
                    count=1
                    j=0
                    
                    
                    for i in b_num:
                        #bc카드 가맹점 입력의 페이지 소스에 해당하는 부분에 사업자 등록번호를 넣어줍니다
                        bc_business_num1 = driver.find_element_by_id('bizregno1')
                        bc_business_num2 = driver.find_element_by_id('bizregno2')
                        bc_business_num3 = driver.find_element_by_id('bizregno3')
                        bc_business_num1.send_keys(i[0:3])
                        bc_business_num2.send_keys(i[3:5])
                        bc_business_num3.send_keys(i[5:10])
                        #입력을 하지도 않았는데 조회버튼을 누르는것을 방지하기위해 timesleep을 걸어주었습니다.
                        time.sleep(1)
                        #조회버튼의 페이지소스에 해당하는 부분을 .click을 이용하여 조회합니다.
                        driver.find_element_by_class_name("center").click()
                        time.sleep(1)
                        
                        #가맹점명을 따옵니다
                        a=driver.find_element_by_id("listTB")
                        b=a.text.split("\n")
                        c.append(str(count)+b[0].split(" ")[2])
                        
                        
                        #가맹점명의 폴더를 생성합니다
                        if(os.path.isdir(dir_path+'/'+dir_name+'/'+c[j])):
                            None
                        else:
                            os.mkdir(dir_path+'/'+dir_name+'/'+c[j])
                            
                        #스크린샷을 찍은뒤 저장합니다
                        driver.save_screenshot('캡쳐/'+nowdate+'/'+c[j]+'/'+str(count)+ '.png')
                        
                        #한번에 많은 사업자등록번호를 입력하였다면 다음 사업자 등록번호 입력을 위해 기존의 사업자등록번호를 지워줍니다.
                        bc_business_num1.clear()
                        bc_business_num2.clear()
                        bc_business_num3.clear()
                        count+=1
                        j+=1
                        
                        #예외처리를 발생시켜 사업자 등록번호를 확인하라는 공고를 줍니다.
                except Exception as e:
                    QMessageBox.about(self,"","사업자 번호를 확인해 주시기 바랍니다.")
                    return 0
                                            
                    # 나머지 부분의 알고리즘도 bc카드와 동일하게 진행 됩니다.
                    # 단 콤보박의가 bc인지 우편인지 ALL인지에 따라 카드사의 종류를 다르게 해놓았습니다.
                    
        elif(self.cb.currentText()=='우편'):
            if(len(num)<10):
                QMessageBox.about(self,"","사업자 번호가 10자리 미만입니다.")
            else:   
                try:
                    QMessageBox.about(self,"","OK 버튼을 누르면 캡쳐를 시작합니다.")
                    b_num=num.split('.')
                    b_num
                    c=[]
                    driver.get('https://www.bccard.com/app/merchant/StoreNoInqActn.do')
                    driver.execute_script("window.scrollTo(0,140);")
                    count=1
                    j=0
                    for i in b_num:
                        bc_business_num1 = driver.find_element_by_id('bizregno1')
                        bc_business_num2 = driver.find_element_by_id('bizregno2')
                        bc_business_num3 = driver.find_element_by_id('bizregno3')
                        bc_business_num1.send_keys(i[0:3])
                        bc_business_num2.send_keys(i[3:5])
                        bc_business_num3.send_keys(i[5:10])
                        driver.find_element_by_class_name("center").click()
                        time.sleep(1)
                        a=driver.find_element_by_id("listTB")
                        b=a.text.split("\n")
                        c.append(str(count)+b[0].split(" ")[2])
                        if(os.path.isdir(dir_path+'/'+dir_name+'/'+c[j])):
                            None
                        else:
                            os.mkdir(dir_path+'/'+dir_name+'/'+c[j])
                        driver.save_screenshot('캡쳐/'+nowdate+'/'+c[j]+'/'+str(count)+ '.png')
                        bc_business_num1.clear()
                        bc_business_num2.clear()
                        bc_business_num3.clear()
                        count+=1
                        j+=1
                        
                except Exception as e:
                    QMessageBox.about(self,"","사업자 번호를 확인해 주시기 바랍니다.")
                    return 0

                driver.get('https://www.samsungcard.com/merchant/number/UHPMMM0101M0.jsp')
                j=0
                for i in b_num:
                    SS_business_num = driver.find_element_by_id('uiInBzrno')
                    SS_business_num.send_keys(i)
                    driver.find_element_by_id("btnSearch").click()
                    time.sleep(1)
                    a=driver.find_element_by_class_name("name")
                    b=a.text.split(' ')[0]
                    driver.save_screenshot('캡쳐/'+nowdate+'/'+c[j]+'/'+str(count)+ '.png')
                    time.sleep(1)
                    SS_business_num.clear()
                    count+=1
                    j+=1
                try:
                    j=0
                    for i in b_num:
                        driver.get('https://www.shinhancard.com/hpe/HPEINFON/mchtNA01List.shc')
                        time.sleep(2)
                        SH_business_num = driver.find_element_by_id('ssn')
                        SH_business_num.send_keys(i)
                        driver.find_element_by_class_name("btn").click()
                        time.sleep(2)
                        a=driver.find_element_by_class_name("textL")
                        b=a.text
                        driver.save_screenshot('캡쳐/'+nowdate+'/'+c[j]+'/'+str(count)+ '.png')
                        time.sleep(1)
                        count+=1
                        j+=1
                except Exception as e :
                    j=0
                    for i in b_num:
                        driver.get('https://www.shinhancard.com/hpe/HPEINFON/mchtNA01List.shc')
                        time.sleep(2)
                        SH_business_num = driver.find_element_by_id('ssn')
                        SH_business_num.send_keys(i)
                        driver.find_element_by_class_name("btn").click()
                        time.sleep(2)
                        a=driver.find_element_by_class_name("textL")
                        b=a.text
                        driver.save_screenshot('캡쳐/'+nowdate+'/'+c[j]+'/'+str(count)+ '.png')
                        time.sleep(1)
                        count+=1
                        j+=1
                    
                driver.get('https://www.hanacard.co.kr/OMA25000000M.web?schID=mcd&mID=OMA25000000M')
                j=0
                for i in b_num:
                    HA_business_num = driver.find_element_by_id('BZNO1')
                    HA_business_num.send_keys(i)
                    driver.find_element_by_class_name('btn_ty02').click()
                    a=driver.find_element_by_class_name("table_ty02")
                    b=a.text.split('\n')
                    d=b[1].split(" ")[0]
                    driver.save_screenshot('캡쳐/'+nowdate+'/'+c[j]+'/'+str(count)+'.png')
                    time.sleep(2)
                    HA_business_num.clear()
                    count+=1
                    j+=1

                driver.get('https://merchant.lottecard.co.kr/app/LMSVCFA_V100.lc')
                j=0
                for i in b_num:
                    LT_business_num = driver.find_element_by_id('rs0Bzno')
                    LT_business_num.send_keys(i)
                    driver.find_element_by_class_name("btnL").click()
                    a=driver.find_element_by_class_name("tblH")
                    b=a.text.split('\n')
                    d=b[1].split(" ")[1]
                    driver.save_screenshot('캡쳐/'+nowdate+'/'+c[j]+'/'+str(count)+'.png')
                    driver.back()
                    count+=1
                    j+=1
            
        else:
            if(len(num)<10):
                QMessageBox.about(self,"","사업자 번호가 10자리 미만입니다.")
            else:   
                try:
                    QMessageBox.about(self,"","OK 버튼을 누르면 캡쳐를 시작합니다.")
                    b_num=num.split('.')
                    b_num
                    c=[]
                    driver.get('https://www.bccard.com/app/merchant/StoreNoInqActn.do')
                    driver.execute_script("window.scrollTo(0,140);")
                    count=1
                    j=0
                    for i in b_num:
                        bc_business_num1 = driver.find_element_by_id('bizregno1')
                        bc_business_num2 = driver.find_element_by_id('bizregno2')
                        bc_business_num3 = driver.find_element_by_id('bizregno3')
                        bc_business_num1.send_keys(i[0:3])
                        bc_business_num2.send_keys(i[3:5])
                        bc_business_num3.send_keys(i[5:10])
                        driver.find_element_by_class_name("center").click()
                        time.sleep(1)
                        a=driver.find_element_by_id("listTB")
                        b=a.text.split("\n")
                        c.append(str(count)+b[0].split(" ")[2])
                        if(os.path.isdir(dir_path+'/'+dir_name+'/'+c[j])):
                            None
                        else:
                            os.mkdir(dir_path+'/'+dir_name+'/'+c[j])
                        driver.save_screenshot('캡쳐/'+nowdate+'/'+c[j]+'/'+str(count)+ '.png')
                        bc_business_num1.clear()
                        bc_business_num2.clear()
                        bc_business_num3.clear()
                        count+=1
                        j+=1
                        
                except Exception as e:
                    QMessageBox.about(self,"","사업자 번호를 확인해 주시기 바랍니다.")
                    return 0;

                driver.get('https://www.samsungcard.com/merchant/number/UHPMMM0101M0.jsp')
                j=0
                for i in b_num:
                    SS_business_num = driver.find_element_by_id('uiInBzrno')
                    SS_business_num.send_keys(i)
                    driver.find_element_by_id("btnSearch").click()
                    time.sleep(1)
                    a=driver.find_element_by_class_name("name")
                    b=a.text.split(' ')[0]
                    driver.save_screenshot('캡쳐/'+nowdate+'/'+c[j]+'/'+str(count)+ '.png')
                    time.sleep(1)
                    SS_business_num.clear()
                    count+=1
                    j+=1

                try:
                    j=0
                    for i in b_num:
                        driver.get('https://www.shinhancard.com/hpe/HPEINFON/mchtNA01List.shc')
                        time.sleep(2)
                        SH_business_num = driver.find_element_by_id('ssn')
                        SH_business_num.send_keys(i)
                        driver.find_element_by_class_name("btn").click()
                        time.sleep(2)
                        a=driver.find_element_by_class_name("textL")
                        b=a.text
                        driver.save_screenshot('캡쳐/'+nowdate+'/'+c[j]+'/'+str(count)+ '.png')
                        time.sleep(1)
                        count+=1
                        j+=1
                except Exception as e :
                    j=0
                    for i in b_num:
                        driver.get('https://www.shinhancard.com/hpe/HPEINFON/mchtNA01List.shc')
                        time.sleep(2)
                        SH_business_num = driver.find_element_by_id('ssn')
                        SH_business_num.send_keys(i)
                        driver.find_element_by_class_name("btn").click()
                        time.sleep(2)
                        a=driver.find_element_by_class_name("textL")
                        b=a.text
                        driver.save_screenshot('캡쳐/'+nowdate+'/'+c[j]+'/'+str(count)+ '.png')
                        time.sleep(1)
                        count+=1
                        j+=1

                driver.get('https://www.hanacard.co.kr/OMA25000000M.web?schID=mcd&mID=OMA25000000M')
                j=0
                for i in b_num:
                    HA_business_num = driver.find_element_by_id('BZNO1')
                    HA_business_num.send_keys(i)
                    driver.find_element_by_class_name('btn_ty02').click()
                    a=driver.find_element_by_class_name("table_ty02")
                    b=a.text.split('\n')
                    d=b[1].split(" ")[0]
                    driver.save_screenshot('캡쳐/'+nowdate+'/'+c[j]+'/'+str(count)+'.png')
                    time.sleep(2)
                    HA_business_num.clear()
                    count+=1
                    j+=1

                driver.get('https://merchant.lottecard.co.kr/app/LMSVCFA_V100.lc')
                j=0
                for i in b_num:
                    LT_business_num = driver.find_element_by_id('rs0Bzno')
                    LT_business_num.send_keys(i)
                    driver.find_element_by_class_name("btnL").click()
                    a=driver.find_element_by_class_name("tblH")
                    b=a.text.split('\n')
                    d=b[1].split(" ")[1]
                    driver.save_screenshot('캡쳐/'+nowdate+'/'+c[j]+'/'+str(count)+'.png')
                    driver.back()
                    count+=1
                    j+=1

                driver.get('https://biz.kbcard.com/CXERFMGC0009.cms')
                driver.execute_script("window.scrollTo(0,240);")
                j=0
                for i in b_num:
                    KB_business_num1 = driver.find_element_by_id('사업자번호1')
                    KB_business_num2 = driver.find_element_by_id('사업자번호2')
                    KB_business_num3 = driver.find_element_by_id('사업자번호3')
                    KB_business_num1.send_keys(i[0:3])
                    KB_business_num2.send_keys(i[3:5])
                    KB_business_num3.send_keys(i[5:10])
                    driver.find_element_by_class_name("action").click()
                    time.sleep(3)
                    a=driver.find_element_by_class_name("tblH")
                    b=a.text.split("\n")[1]
                    d=b.split(" ")[1]
                    driver.save_screenshot('캡쳐/'+nowdate+'/'+c[j]+'/'+str(count)+'.png')
                    KB_business_num1.clear()
                    KB_business_num2.clear()
                    KB_business_num3.clear()
                    count+=1
                    j+=1

                driver.get('https://www.hyundaicard.com/csa/mb/STOREMAIN.hc')
                time.sleep(1)
                driver.find_element_by_id("searchPopup").click()
                j=0
                for i in b_num:
                    HD_business_num = driver.find_element_by_id('popFormLSN')
                    HD_business_num.send_keys(i)
                    driver.find_element_by_id("pop_gmjPopup_btn").click()
                    time.sleep(1)
                    a=driver.find_element_by_class_name("list_area")
                    b=a.text.split('\n')[4]
                    driver.save_screenshot('캡쳐/'+nowdate+'/'+c[j]+'/'+str(count)+'.png')
                    HD_business_num.clear()
                    count+=1
                    j+=1



                driver.get('https://ibz.nonghyup.com/servlet/ICCV1420I.view')        

            
        
        


if __name__ == "__main__" : 
    app = QApplication(sys.argv)
    mywindow = MyWindow()
    mywindow.show()
    app.exec_()
    




C:\Users\LG\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: DeprecationWarning: use options instead of chrome_options


## 제작이유 : 대구은행 인턴중 가맹점 결제계좌변경 서류에 가맹점 번호가 올바르지 기입되지않으면 안되는데 확인을 하기위해 모든 카드사에 일일이 접속해서 가맹점 번호를 치고 있는것이 효율적이지 않다고 생각 하여서 하나의 사업자 등록번호를 입력만 한다면 모든카드사의 가맹점 번호를 캡쳐 하는 프로그램을 작성하였습니다.